In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

import transformers
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler

import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-04-07 18:03:06.762936: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 18:03:06.763035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 18:03:06.890637: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/sarcasm-detection/task_B_En_test.csv
/kaggle/input/sarcasm-detection/task_A_Ar_test.csv
/kaggle/input/sarcasm-detection/train.En.csv
/kaggle/input/sarcasm-detection/task_C_Ar_test.csv
/kaggle/input/sarcasm-detection/arabic_task_a.csv
/kaggle/input/sarcasm-detection/english_task_a.csv
/kaggle/input/sarcasm-detection/arabic_task_c.csv
/kaggle/input/sarcasm-detection/task_C_En_test.csv
/kaggle/input/sarcasm-detection/task_A_En_test.csv
/kaggle/input/sarcasm-detection/train.Ar.csv
/kaggle/input/sarcasm-detection/english_task_c.csv
/kaggle/input/sarcastic-comments-on-reddit/train-balanced-sarcasm.csv


# TASK 1

In [2]:
train= pd.read_csv("/kaggle/input/sarcasm-detection/train.En.csv")
print(train.shape)

(3468, 10)


In [3]:
train.head()

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train.isna().sum()

Unnamed: 0                0
tweet                     1
sarcastic                 0
rephrase               2601
sarcasm                2601
irony                  2601
satire                 2601
understatement         2601
overstatement          2601
rhetorical_question    2601
dtype: int64

In [5]:
simple_df = train[["tweet","sarcastic"]]
simple_df

,tweet,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1
...,...,...
3463,The population spike in Chicago in 9 months is...,0
3464,You'd think in the second to last English clas...,0
3465,I’m finally surfacing after a holiday to Scotl...,0
3466,Couldn't be prouder today. Well done to every ...,0


In [6]:
df = train.dropna()
df

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
862,862,yo @claires do yall do hysterectomies?,1,"Claires, you should not do full hysterectomies.",1.0,0.0,0.0,0.0,0.0,1.0
863,863,@JacobWohlReport Do I need to aquire a wife be...,1,A lot of people don't have a wife.,1.0,0.0,0.0,0.0,0.0,1.0
864,864,I get a lot of boy who cried wolf vibes from t...,1,The red cross is always needy.,0.0,1.0,0.0,0.0,0.0,0.0
865,865,Update: holding hands with your mom and walkin...,1,Holding hands with your parent while walking a...,1.0,0.0,0.0,0.0,0.0,0.0


- L'idee de task 1 est de finetune a pre-trained model sur plus de donnees
- en task 2 en fait augmentation, pas de choix !!

In [7]:
simple_df.isna().sum()

tweet        1
sarcastic    0
dtype: int64

In [8]:
simple_df = simple_df.dropna()
simple_df = simple_df.reset_index(drop = True)


In [9]:
simple_df['sarcastic'].value_counts()

sarcastic
0    2600
1     867
Name: count, dtype: int64

In [10]:
simple_df['tweet'][0]

'The only thing I got from college is a caffeine addiction'

#### enlever les liens

In [11]:
import re
def remove_links(text):
    # Expression régulière pour trouver et enlever les liens
    link_pattern = re.compile(r'http\S+|www\S+')
    return link_pattern.sub(r'', text)

simple_df['tweet'] = simple_df['tweet'].apply(remove_links)


#### Enlever les tags 

In [12]:
def remove_tags(text):

    return re.sub(r'@\w+', '', text)

simple_df['tweet'] = simple_df['tweet'].apply(remove_tags)


#### enlever les hashtags

In [13]:
import re

def extract_hashtags(df, text_column_name, new_column_name):
    # Create an empty list to store extracted hashtags
    hashtags_list = []
    
    # Regular expression pattern to match hashtags
    hashtag_pattern = r'#\w+'
    
    # Loop through each row in the DataFrame
    for text in df[text_column_name]:
        # Find all hashtags in the text using regular expression
        hashtags = re.findall(hashtag_pattern, text)
        # Append the list of hashtags to the hashtags_list
        hashtags_list.append(hashtags)
    
    # Add the hashtags_list as a new column in the DataFrame
    df[new_column_name] = hashtags_list
    
    return df

def remove_hashtags(text):
    cleaned_text = re.sub(r'#\w+', '', text)
    return cleaned_text



simple_df = extract_hashtags(simple_df, 'tweet', 'Hashtags')

simple_df['tweet'] = simple_df['tweet'].apply(remove_hashtags)

In [14]:
simple_df['Hashtags'].value_counts()

Hashtags
[]                                                 3087
[#AEWAllOut]                                          2
[#GhostOfTsushima, #PS5Share, #GhostofTsushima]       2
[#matthanock]                                         2
[#evermorealbum]                                      2
                                                   ... 
[#newbie]                                             1
[#grav3yardgirl]                                      1
[#ww111]                                              1
[#BrexitShambles]                                     1
[#SpaceX]                                             1
Name: count, Length: 365, dtype: int64

#### enlever les caractères spéciaux 

In [15]:
import re

# Fonction pour enlever les caractères spéciaux
def remove_special_chars(text):
    # Expression régulière pour trouver les caractères spéciaux 
    special_chars_pattern = re.compile(r'[^\w\s]')
    return re.sub(special_chars_pattern, '', text)

# Appliquer la fonction remove_special_chars() à la colonne 'Text'
simple_df['tweet'] = simple_df['tweet'].apply(remove_special_chars)

#### enlever les tabulations \t et new lines \n

In [16]:
def remove_spaces_and_newlines(text):
    # Remplace les espaces, les tabulations et les sauts de ligne par une chaîne vide
    return text.replace("\t", "").replace("\n", "")

simple_df['tweet'] = simple_df['tweet'].apply(remove_spaces_and_newlines)

#### enlever les emojis

In [17]:
def extract_emojis(text):
    emoji_pattern = re.compile("["
                             u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)


simple_df['Emojis'] = simple_df['tweet'].apply(extract_emojis)

#simple_df


In [18]:
#simple_df['Emojis'].value_counts()

## DeBerta

In [18]:
from transformers import AutoModel, DebertaV2Tokenizer

model = AutoModel.from_pretrained("microsoft/DeBERTa-V3-XSmall")

tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/DeBERTa-V3-XSmall")
tokenized_data = tokenizer(simple_df['tweet'].tolist(), padding=True, truncation=True, return_tensors='pt')


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [19]:
input_ids = tokenized_data.input_ids
attention_mask = tokenized_data.attention_mask

In [21]:
batch_size = 32
import torch

# Découper les données en batches
input_ids_batches = torch.split(input_ids, batch_size, dim=0)
attention_mask_batches = torch.split(attention_mask, batch_size, dim=0)

outputs = []
for batch_input_ids, batch_attention_mask in zip(input_ids_batches, attention_mask_batches):
    batch_output = model(batch_input_ids, attention_mask=batch_attention_mask)
    outputs.extend(batch_output.last_hidden_state.tolist())
    

In [33]:
import pickle

# Enregistrer la variable outputs dans un fichier
with open("/kaggle/working/DeBerta_embeddings.pkl", "wb") as f:
    pickle.dump(outputs, f)

In [34]:
with open ("/kaggle/working/DeBerta_embeddings.pkl", "rb") as f:
    embedding = pickle.load(f)

## Roberta

In [19]:
from transformers import RobertaTokenizer, AutoModelForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/roberta-base")


In [20]:
# Calculer la longueur maximale des tweets dans simple_df['tweet']
max_tweet_length = simple_df['tweet'].apply(lambda x: len(x.split())).max()

# Utiliser la longueur maximale pour le padding
tokenized_data = tokenizer(simple_df['tweet'].tolist(), padding='max_length', max_length=max_tweet_length, truncation=True, return_tensors='pt')


In [21]:
input_ids = tokenized_data.input_ids
attention_mask = tokenized_data.attention_mask

In [ ]:
batch_size = 8
import torch

# Découper les données en batches
input_ids_batches = torch.split(input_ids, batch_size, dim=0)
attention_mask_batches = torch.split(attention_mask, batch_size, dim=0)

outputs = []
for batch_input_ids, batch_attention_mask in zip(input_ids_batches, attention_mask_batches):
    outputs.extend(model(batch_input_ids, attention_mask=batch_attention_mask).logits.tolist())
    print(len(outputs))
    

8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224
232
